In [119]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/bangkok_traffy.csv')

In [120]:
pd.set_option('display.max_colwidth', None)

In [121]:
df.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_bucket/attachment/2021-09/3063e748259afbb7171467e19b92e9cc1f1a5826.jpg,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆปีจะมีเครื่องสูบน้ำแต่ปีนี้ไม่มีกทม.ปล่อยทิ้ง ชุมชนเคหะนคร1แปลง2(ซ.เฉลิมพระเกียรติร.9ซอง22 วัดตะกล่ำ ประเวศ),https://storage.googleapis.com/traffy_public_bucket/attachment/2021-09/41ef2b1b465b708db17363a6d5fd8c5336266df7.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2022-06/b562e36f971f02221bd0f32b7272eb1ea516cacf.jpg,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเวศ กรุงเทพมหานคร 10250 ประเทศไทย,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_bucket/attachment/2021-09/b0b738c84fb0d0b0bfd5c5ac004492eb6fde19c3.jpg,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเทพมหานคร 10500 ประเทศไทย,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_bucket/attachment/2021-10/5a36fa060eb2a750e3ff47cdd30307f4639f445e.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2022-09/7739f2143dfff32905f68fbf7b11537e1a7b447d.jpg,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบางซื่อ กรุงเทพมหานคร 10800 ประเทศไทย,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว\n\n1. หลังจากฝนตกต่อเนื่องประมาณ 30 นาที น้ำท่วมซอยสูงประมาณ 20 ซม ใช้เวลาระบายราวๆ 3-5 ชม. กว่าน้ำในซอยแห้ง\nหากฝนตกนานกว่า 1 ชม ใช้เวลา 2-3 วัน กว่าน้ำในซอยแห้ง\nหากนานกว่านั้น ต้นซอยน้ำแห้ง ปลายซอยน้ำเน่า,https://storage.googleapis.com/traffy_public_bucket/attachment/2021-12/6b71a10f77a724a6151180eb5eb5bb0fc1500ce9.jpg,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว กรุงเทพมหานคร 10230 ประเทศไทย,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


In [122]:
footpath_keywords = [
    "ทางเท้า","ฟุตบาท","พื้นผิว","ฝาท่อ","แผ่นพื้น",
    "พื้นแตก","แตก","รอยแตก","หลุม","บ่อ",
    "พื้นทางเดิน","กระเบื้อง","แผ่นทางเท้า"
]

pattern = "|".join(footpath_keywords)

df_fp = df[
    df["type"].astype(str).str.contains(pattern, case=False, na=False) |
    df["comment"].astype(str).str.contains(pattern, case=False, na=False)
]

In [123]:
df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
df_fp["lon"] = pd.to_numeric(df_fp["lon"], errors="coerce")
df_fp["lat"] = pd.to_numeric(df_fp["lat"], errors="coerce")

C:\Users\Karin\AppData\Local\Temp\ipykernel_27184\2938595523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
C:\Users\Karin\AppData\Local\Temp\ipykernel_27184\2938595523.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp[["lon","lat"]] = df_fp["coords"].str.split(",", expand=True)
C:\Users\Karin\AppData\Local\Temp\ipykernel_27184\2938595523.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [124]:
df_fp["timestamp"] = pd.to_datetime(df_fp["timestamp"], errors="coerce")
df_fp["last_activity"] = pd.to_datetime(df_fp["last_activity"], errors="coerce")

C:\Users\Karin\AppData\Local\Temp\ipykernel_27184\300353576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp["timestamp"] = pd.to_datetime(df_fp["timestamp"], errors="coerce")
C:\Users\Karin\AppData\Local\Temp\ipykernel_27184\300353576.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp["last_activity"] = pd.to_datetime(df_fp["last_activity"], errors="coerce")


In [125]:
df_fp["duration_hours"] = (
    df_fp["last_activity"] - df_fp["timestamp"]
).dt.total_seconds() / 3600

df_fp["duration_hours"].describe()

C:\Users\Karin\AppData\Local\Temp\ipykernel_27184\840720600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fp["duration_hours"] = (


count    210176.000000
mean       1727.335198
std        3348.604418
min          -0.000399
25%          40.425008
50%         186.452099
75%        1577.000691
max       25121.953528
Name: duration_hours, dtype: float64

In [ ]:
df_fp = df_fp[df_fp["duration_hours"] > 0]

df_fp = df_fp[df_fp["state"] =='เสร็จสิ้น']

lower_bound = df_fp["duration_hours"].quantile(0.01)
upper_bound = df_fp["duration_hours"].quantile(0.99)

df_fp["duration_hours"] = df_fp["duration_hours"].clip(lower=lower_bound, upper=upper_bound)
df_fp["duration_hours"].describe()

print(lower_bound)

1.34777204875


In [127]:
pd.set_option('display.max_colwidth', None)

In [128]:
df_fp.sort_values(by="duration_hours", ascending=True, inplace=True)
df_fp.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,lon,lat,duration_hours
170250,2022-C8HTMV,"{น้ำท่วม,คลอง,ท่อระบายน้ำ,ถนน}","เขตบางเขน,ผอ.เขตบางเขน (น.ส.อัญชนา),กลุ่มกรุงเทพเหนือ (นางวันทนีย์ วัฒนะ)",ขอปรับปรุงยกระดับถนนแยก22 (รามอินทรา34) เดิม ซอยรุ่งสุวรรณ เขตบางกะปิ&เขตลาดพร้าว\nตามที่เขตบางเขนบ่ายเบี่ยงว่าไม่เป็นที่สาธารณะขอให้ไปตรวจสอบที่สำนักงานที่ดินเขตลาดพร้าวได้ หมู่บ้านนนทรี ซื้อสิทธิ์ทางออก จดทะเบียนเป็นถนนสาธารณะประมาณ 40 ปี เคยทำเรื่องผ่านศูนย์ กทม.1555 หลายครั้งหลายหน ทั้งเขตลาดพร้าว -บางเขน ผ่าน สส. สก. จนปี 46 ได้งบผ่าถนน ฝังท่อระบายน้ำ ฝนตกน้ำท่วมทุกปี ไม่เคยมีใครสนใจดูแลเลยครับ โครงการทำเขื่อนคลองโคกคราม คลองตาเริง ประตูน้ำคลองบางขวด รถบรรทุก 10ล้อ รถเทเลอร์ ขนรถแม็คโคร เหล็กเส้น เหล็กตัวไอ แผ่นปูนสำเร็จ ขนดินคลองออก ทำถนนแตกร้าวตลอดซอย รับว่าจะซ่อมให้ พอเสร็จโครงการทิ้งเลยไม่ทำตามที่ตกลงจึง ขอให้เขตบางเขนดำเนินการปรับปรุงถนนเป็นถนนคอนกรีตพร้อมท่อระบายน้ำแยก 22 รามอินทรา 34 ยกสูงเท่าถนนรามอินทรา34 (เท่าแยก 19) พร้อมปาดผายทางเข้าออกฝนตกน้ำท่วมขังทุกปี จึงเรียนมาเพื่อโปรดดำเนินการ,https://storage.googleapis.com/traffy_public_bucket/attachment/2022-11/df3284d88bc91cdc5f4965a0a4d8a39a7d1c2336.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2022-11/d8816724ba7af06d4a57acde0d8bc3f5ace835b7.jpg,"100.63212,13.84398",5 ซอย รามอินทรา 34 แยก 22 แขวง ท่าแร้ง เขตบางเขน กรุงเทพมหานคร 10220 ประเทศไทย,ท่าแร้ง,บางเขน,กรุงเทพมหานคร,2022-11-08 07:46:59.997853+00:00,เสร็จสิ้น,4.0,0,2022-11-08 08:36:32.091837+00:00,100.63212,13.84398,1.347772
629121,2024-73KX38,{ทางเท้า},"เขตสัมพันธวงศ์,ฝ่ายโยธา เขตสัมพันธวงศ์",มีการก่อสร้างปิดทางเดินเท้าแบบสนิด ไม่มั่นใจว่ามีการขออนุญาตหรือไม่,https://storage.googleapis.com/traffy_public_bucket/attachment/2024-07/e99bf7a135c53c5352175ef5073e3eab05d345aa.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2024-07/815a8caa8a1cd56bca46120bf8180e6f.png,"100.50647,13.74153",141 Ratchawong Rd แขวงจักรวรรดิ์ เขตสัมพันธวงศ์ กรุงเทพมหานคร 10100 ประเทศไทย,จักรวรรดิ,สัมพันธวงศ์,กรุงเทพมหานคร,2024-07-17 06:23:28.977901+00:00,เสร็จสิ้น,1.0,0,2024-07-17 07:35:12.839711+00:00,100.50647,13.74153,1.347772
562875,2024-9ZPULW,{กีดขวาง},เขตสาทร,เรียนท่านผู้ว่าก.ท.ม\nถึงก่อนปากซอยจันทร์18/7ใกล้ป้อมตำรวจจราจรมีรถจักรยานยนต์ขี่จอดบนทางเท้าทะเบียนรถฬตกกรุงเทพมหานคร627ช่วงเวลา20:00น.เขตสาทร,https://storage.googleapis.com/traffy_public_bucket/attachment/2024-05/a92b5c9477a4e8faab212dd0460b5f1252e5eeb9.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2024-05/791c3400d89cea296cfc368bcaf9a0a8.png,"100.52604,13.70655",336 ถ. จันทน์ แขวงทุ่งวัดดอน เขตสาทร กรุงเทพมหานคร 10120 ประเทศไทย,ยานนาวา,สาทร,กรุงเทพมหานคร,2024-05-06 13:45:24.294499+00:00,เสร็จสิ้น,NaN,0,2024-05-06 13:59:53.364460+00:00,100.52604,13.70655,1.347772
784919,2025-6XCBXR,{ทางเท้า},"เขตบางเขน,ฝ่ายเทศกิจ เขตบางเขน",รถมอเตอร์ไซค์วิ่งบนทางเท่า,https://storage.googleapis.com/traffy_public_bucket/attachment/2025-01/880ab7e61ba24d5594ec0fa97b7ef29f22696d77.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2025-01/846e5bf5582f59d4c6f3505349767eb1.png,"100.58995,13.87933",3 ถ. แจ้งวัฒนะ แขวงอนุสาวรีย์ เขตบางเขน กรุงเทพมหานคร 10220 ประเทศไทย,อนุสาวรีย์,บางเขน,กรุงเทพมหานคร,2025-01-14 01:00:19.145338+00:00,เสร็จสิ้น,NaN,0,2025-01-14 02:19:33.534232+00:00,100.58995,13.87933,1.347772
266958,2023-AHYKHW,{},เขตจอมทอง,พบกระจกแยกทางโค้งแตกหัก ชำรุด,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-05/83a7b068550b5a547dd3ec12646b347249daf13b.jpg,https://storage.googleapis.com/traffy_public_bucket/attachment/2023-05/1683461055762.jpeg,"100.47460,13.67760",60/4 พระรามที่ 2 ซอย 25 แขวง บางมด เขตจอมทอง กรุงเทพมหานคร 10150 ประเทศไทย,บางมด,จอมทอง,กรุงเทพมหานคร,2023-05-07 11:43:23.147016+00:00,เสร็จสิ้น,5.0,0,2023-05-07 12:04:17.196520+00:00,100.47460,13.67760,1.347772
